In [1]:
from matplotlib import *
import numpy as np
import scipy
from scipy.integrate import odeint, ode, complex_ode
from cmath import *
from pylab import *

# Classes definitions

In [2]:
class TWPA:
    #this guy contains TWPA parameters and can provide one with 
    #ODE functions for P,S,I.
    #All manipulations are to be made in the other class
    
    ##########################################
    #definitions and parameters set functions#
    ##########################################
    
    #Line parameters per unit length
    C = 4.5e-14
    L = 3.29e-14
    alpha = 10e-6 #=1/L_*^2
    R = 1e-3
    G = 1e-4
    
    LineLength = 2 #m, spiral length
    
    #######################################
    #Freqs and waveVectors
    wp = 6.28e9 * 6.
    ws = 6.28e9 * 7.
    wi = 1e0
    
    kp = 1e1
    ks = 1e1
    ki = 1e1
    deltaK = 1e1
    
    #Current in line
    A0 = [1e1,1e1,1e1,0.,0.,0.]  #initial condition, Ap,As,Ai,thetaP,thetaS,thetaI
    A = [1e1,1e1,1e1,0.,0.,0.]   #not needed any more. Too lazy to remove though
    #
    
    def __init__(self):
        self.L = 1e-10
        self.C = 1e-15
        self.R = 0.
        self.G = 0.
        self.alpha = 1e-4
        self.LineLength = 2.
        self.wp = 6*6.28e9
        self.ws = 6*6.28e9

    ##################
    #Set functions####
    ##################
    def setC(self, x):
        self.C = x
        self.WaveVectors()
        
    def setL(self, x):
        self.L = x
        self.WaveVectors()
        
    def setR(self, x):
        self.R = x
        self.WaveVectors()
        
    def setG(self, x):
        self.G = x
        self.WaveVectors()
    
    def setAlpha(self, x):
        self.alpha = x
        self.WaveVectors()
        
    def setLineLength(self, x):
        self.LineLength = x
    
    #################
    
    def setWp(self, x):
        self.wp = x
        self.WaveVectors()
        
    def setWs(self, x):
        self.ws = x
        self.wi = 2*self.wp - self.ws
        self.WaveVectors()
        
    def WaveVectorZeroth(self,w_input): ###Careful - перенесено на другую строку
        return( (  w_input**2 * self.C*self.L - self.G *self.R  )**0.5 )
    
    def WaveVectors(self):
        self.kp = self.WaveVectorZeroth(self.wp)
        self.ks = self.WaveVectorZeroth(self.ws)
        self.ki = self.WaveVectorZeroth(self.wi)
        self.deltaKl = -2*self.kp + self.ks + self.ki
        
    ################
    
    def setA0(self,l):
        if(len(l)!=6):
            print("wrong y0 array length \n")
            return(-1)
        self.A0 = l
        
    def setA(self,l):
        if(len(l)!=6):
            print("wrong Y array length \n")
            return(-1)
        self.A = l
        
    def AtoA0(self):
        self.A = np.copy(self.A0)
        
    #############
    #Just for userfriendliness
    def printState(self):
        print 'C =', self.C
        print 'L =', self.L
        print 'R =', self.R
        print 'G =', self.G
        print 'alpha =', self.alpha
        print 'Line length =', self.LineLength, 'm'
        print '\n'
        print 'Pump freq =', self.wp/6.28e9, 'GHz'
        print 'Signal freq =', self.ws/6.28e9, 'GHz'
        print 'Idler freq =', self.wi/6.28e9, 'GHz'
        print 'kp =', self.kp
        print 'ks =', self.ks
        print 'ki =', self.ki
        print '\n'
        print 'A = (Ap,As,Ai,thetaP,thetaS,thetaI)'
        print 'A0 = ', self.A0, 
        print 'A = ', self.A
        
    ###########################
    ### ODE part begins here###
    ###########################
    
    def getGamma(self,w_input):
        return(self.alpha * self.L *w_input / 8 / self.WaveVectorZeroth(w_input))
    
    def getPhase(self,x,q):#Phase shifting is to be added here
        return(q[4]+ q[5] - 2*q[3] + self.deltaK * x)
        #return pi/2.
    
    def FuncPump(self,x,q):#Returns [amplitude, phase]. q = [ap,as,ai, theta_p,..,..]
        ####################
        a =  - 0.5*self.wp*(self.C * self.R + self.G * self.L)* q[0] / self.kp - self.getGamma(self.wp)*self.wp*self.C* 2*q[1]*q[2]*q[0]*np.sin(self.getPhase(x,q)) - self.getGamma(self.wp)*self.G * ( q[0] * (q[0]**2 + 2*q[1]**2 + 2*q[2]**2)  + 2*q[0]*q[1]*q[2]*np.cos(self.getPhase(x,q)))
        
        theta = self.getGamma(self.wp)*( self.wp*self.C *( (q[0]**2 + 2*q[1]**2 + 2*q[2]**2)  +  2*q[1]*q[2]*np.cos(self.getPhase(x,q)) )   - self.G* 2*q[1]*q[2] * np.sin(self.getPhase(x,q)))
        return([a,theta])

    def FuncSignal(self,x,q):
        a =  - 0.5*self.ws*(self.C * self.R + self.G * self.L)* q[1] / self.ks + self.getGamma(self.ws)*self.ws*self.C* q[0]**2*q[2]*np.sin(self.getPhase(x,q)) - self.getGamma(self.ws)*self.G * ( q[1] * (2*q[0]**2 + q[1]**2 + 2*q[2]**2)  +  q[0]*q[0]*q[2]*np.cos(self.getPhase(x,q)))
        theta = self.getGamma(self.ws)*( self.ws*self.C *( (2*q[0]**2 + q[1]**2 + 2*q[2]**2)  +   q[0]**2*q[2]/q[1]*np.cos(self.getPhase(x,q)) )  + self.G * q[0]**2 * q[2]/q[1] * np.sin(self.getPhase(x,q)))
        return([a,theta])
     
    def FuncIdler(self,x,q):
        a =  - 0.5*self.wi*(self.C * self.R + self.G * self.L)* q[2] / self.ki + self.getGamma(self.wi)*self.wi*self.C* q[0]**2*q[1]*np.sin(self.getPhase(x,q)) - self.getGamma(self.wi)*self.G * ( q[2] * (2*q[0]**2 + 2*q[1]**2 + q[2]**2)  +  q[0]*q[0]*q[1]*np.cos(self.getPhase(x,q)))
        theta = self.getGamma(self.wi)*( self.wi*self.C *( (2*q[0]**2 + 2*q[1]**2 + q[2]**2)  +   q[0]**2*q[1]/q[2]*np.cos(self.getPhase(x,q)) )  + self.G * q[0]**2 * q[1]/q[2] * np.sin(self.getPhase(x,q)))
        return([a,theta])

    def Function(self,x,q):#returns derivative of array. array of (a_p, a_s, a_i, thetaP, thetaS, thetaI) is on onput
        a_p,theta_p = self.FuncPump(x,q)
        a_s, theta_s = self.FuncSignal(x,q)
        a_i, theta_i = self.FuncIdler(x,q)
        b = np.array([a_p, a_s, a_i,theta_p,theta_s,theta_i])
        return(b)
    

In [3]:
class TWPAmanipulator():
    #this guy can plot all graphs.
    #All functions are to be added here
    
    ampl = TWPA()
    r = ode(ampl.Function).set_integrator('dop853')
    
    w = linspace(0.,1.,1e1)
    wStart = 0.
    wStop = 1.
    wLen = 3e1
    
    spaceGrid = linspace(0.,2.,10)
    
    ##############
    ### Setters###
    
    def __init__(self):
        self.wStart = 4*6.28e9
        self.wStop = 10*6.28e9
        self.wLen = 3e2
        self.w = linspace(self.wStart,self.wStop,self.wLen)
        self.spaceGrid = linspace(0.,2.,3e2)
    
    def setSpaceGridLen(self,x):
        self.spaceGrid = linspace(0.,self.ampl.LineLength,x)
        
    def setWgrid(self):
        self.w = linspace(self.wStart,self.wStop,self.wLen)
    
    def setWstart(self, x): #takes FREQUENCY - in GHz
        self.wStart = x*6.28e9
        self.setWgrid()
        
    def setWstop(self, x):
        self.wStop = x*6.28e9
        self.setWgrid()
        
    def setWlen(self, x):
        self.wLen = x
        self.setWgrid()
        
    def printGrids(self):
        print 'Ws grid: (',self.wStart,',',self.wStop,',',self.wLen,')'
        print 'Space grid: (0,',self.ampl.LineLength,',',len(self.spaceGrid),')'
        
    ##############
    ###Plotters###
    ##############
    
    #!!!!!!!!!!!!!!!!!!!!
    #In case one wants to get results with phase shifting he is just to change output
    # of function TWPA.getPhase() to return(pi/2)
    #!!!!!!!!!!!!!!!!!!!!
    
    
    def GainFromFreq(self): #2d plot
        T = self.spaceGrid[len(self.spaceGrid) - 1]
        t = []
        dt = self.spaceGrid[1] - self.spaceGrid[0]
        gain = []
        idler = []
        difference = []
        theta = []
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            self.r.set_initial_value(self.ampl.A0, 0)
            while self.r.successful() and self.r.t <= T:
                self.r.integrate(self.r.t + dt)
                #t = np.append(t,r.t)
            #gain.append(20*np.log10(self.r.y[1]/self.ampl.y0[1]))
            #idler.append(20*np.log10(self.r.y[2]/self.ampl.y0[1]))
            
            
            #gain.append(20*np.log10(self.r.y[1]/self.ampl.A0[1]))
            gain.append((self.r.y[1]/self.ampl.A0[1]))
            idler.append((self.r.y[2]/self.ampl.A0[2]))
            theta.append(self.r.y[4] + self.r.y[5] - 2*self.r.y[3])
            
            
            #theta.append(2*self.r.y[3] - self.r.y[4] - self.r.y[5] - self.ampl.deltaKl*T)
            #idler.append(20*np.log10(self.r.y[2]/self.ampl.y0[1]))
            #difference.append((self.r.y[1]/self.ampl.y0[1])**2-(self.r.y[2]/self.ampl.y0[1])**2)
            
        plot(self.w/6.28e9,gain,label="Signal gain")
        plot(self.w/6.28e9,idler,label="Idler gain")
        plot(self.w/6.28e9,theta,label="Theta, rads")
        legend()
        xlabel('Frequency, GHz')
        ylabel('Gain, dB')
        show()
        return([gain,idler,theta])
    
    
    ###
    def GainFromFreqAndN(self):
        T = self.spaceGrid[len(self.spaceGrid) - 1]
        t = []
        dt = self.spaceGrid[1] - self.spaceGrid[0]
        gain = []
        idler = []
        theta = []
        k = 0
        for x in linspace(self.wStart,self.wStop,self.wLen):
            self.ampl.setWs(x)
            self.r.set_initial_value(self.ampl.A0, 0)
            gain.append([])
            idler.append([])
            theta.append([])
            while self.r.successful() and (self.r.t <= T):
                self.r.integrate(self.r.t + dt)
                
                gain[k].append(20*np.log10(self.r.y[1]/self.ampl.A0[1]))
                #gain[k].append(self.r.y[1]/self.ampl.A0[1])
                idler[k].append(20*np.log10(self.r.y[2]/self.ampl.A0[2]))
                theta[k].append((self.r.y[4] + self.r.y[5] - 2*self.r.y[3]))
                t = np.append(t,self.r.t)
            
            ###костыль!!!!!Нужен, ибо иногда масссив обрывается. Но может нагадить в данные
            while (len(gain[k]) < len(self.spaceGrid) ):
                gain[k].append(0)
                idler[k].append(0)
                theta[k].append(0)
                
            k = k + 1     
        
        contourf( self.spaceGrid, self.w/6.28e9, gain, 100, alpha=0.75, cmap='jet', vmin = 0.)
        colorbar()
        xlabel("Length of line, m")
        ylabel("Freq, GHz")
        show()
        print ndim(gain)
        return([gain,idler,theta])

# Calculations

In [ ]:
b = TWPAmanipulator()

In [4]:
#Setting parameters
b.ampl.setC(1e-12)
b.ampl.setL(1e-7)
b.ampl.setAlpha(1e12)
b.ampl.setR(0.)
b.ampl.setG(0.)
b.ampl.setLineLength(4)

b.ampl.setWp(6*6.28e9)
b.ampl.setWs(7*6.28e9)

b.ampl.setA0([8e-7,1e-12,1e-16,0.,0.,0.])
b.ampl.printState()
print 'Pump/Crit', (b.ampl.A0[0]**2 * b.ampl.alpha)**0.5
print 'Impedance', (b.ampl.L/b.ampl.C)**0.5

C = 1e-12
L = 1e-07
R = 0.0
G = 0.0
alpha = 1e+12
Line length = 4 m


Pump freq = 6.0 GHz
Signal freq = 7.0 GHz
Idler freq = 5.0 GHz
kp = 11.9154622235
ks = 13.9013725941
ki = 9.92955185293


A = (Ap,As,Ai,thetaP,thetaS,thetaI)
A0 =  [8e-07, 1e-12, 1e-16, 0.0, 0.0, 0.0] A =  [10.0, 10.0, 10.0, 0.0, 0.0, 0.0]
Pump/Crit 0.8
Impedance 316.227766017


In [5]:
##############
#Getting data#
##############


#Setting grids - space and freq ones
b.setWstart(4.)
b.setWstop(8.)
b.setWlen(3e2)

b.setSpaceGridLen(2e3)
#b.printGrids()

#g = b.GainFromFreq()
g = b.GainFromFreqAndN()

2


In [6]:
#Potting 3d
#g[0] - gain, g[1] - idler, g[2] - theta
contourf( b.spaceGrid, b.w/6.28e9, g[2], 100, alpha=0.75, cmap='jet', vmin = 0.)
colorbar()
xlabel("Length of line, m")
ylabel("Freq, GHz")
show()